In [1]:
import secretflow as sf

In [2]:
edge_parties_number = 2
edge_party_name = 'edge_party_{i}'
edge_parties = [edge_party_name.format(i=i) for i in range(edge_parties_number)]
server_party_name = 'server_party'
server_party = [server_party_name]
all_parties = edge_parties + server_party

In [3]:
edge_parties

['edge_party_0', 'edge_party_1']

In [4]:
sf.init(parties=all_parties, address='local')

/home/zoupeicheng.zpc/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-26 15:59:14,566	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-11-26 15:59:14,763	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/home/zoupeicheng.zpc/miniconda3/envs/py310/lib/python3.10/subprocess.py:1796: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _posixsubprocess.fork_exec(
2024-11-26 15:59:16,700	INFO worker.py:1724 -- Started a local Ray instance.


In [5]:
edge_devices = [sf.PYU(edge_party_name.format(i=i)) for i in range(edge_parties_number)]
# use pyu to simulate teeu
edge_tees = [sf.PYU(edge_party_name.format(i=i)) for i in range(edge_parties_number)]

server_device = sf.PYU(server_party_name)
server_tee = sf.PYU(server_party_name)

In [ ]:
# custom parameters
i = 0
j = 1
m = 100
kappa = 32
u_low = 0.0
u_high = 2.0
# k is the ring size 2^k. usually take k = 32, 64. like size of int
k = 32  # implies use uint32
fxp = 26  # fixed point precision

In [7]:
import jax
import jax.numpy as jnp
import numpy as np
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from typing import Tuple
import hashlib


def bytes_to_jax_random_key(byte_key):
    # Hash the bytes key using SHA-256
    hash_digest = hashlib.sha256(byte_key).digest()

    # Convert the first 4 bytes of the hash digest to an integer
    # JAX random keys expect a 32-bit integer, hence we use the first 4 bytes
    seed = int.from_bytes(hash_digest[:4], 'big')

    # Create a JAX random key with this seed
    jax_key = jax.random.PRNGKey(seed)

    return jax_key


# Function to encrypt a jnp array using AES-GCM
def encrypt_jnp_array_gcm(jnp_array, key) -> Tuple[bytes, bytes, bytes]:
    # Convert jnp array to numpy array
    np_array = np.array(jnp_array)

    # Convert numpy array to bytes
    array_bytes = np_array.tobytes()

    # Create AES cipher in GCM mode
    cipher = AES.new(key, AES.MODE_GCM)

    # Encrypt data
    ciphertext, tag = cipher.encrypt_and_digest(array_bytes)

    # Return the ciphertext, tag, and nonce
    return ciphertext, tag, cipher.nonce


# Function to decrypt a jnp array using AES-GCM
def decrypt_to_jnp_array_gcm(ciphertext, tag, nonce, key, dtype, shape):
    # Create AES cipher in GCM mode with the same parameters
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)

    # Decrypt data
    decrypted_data = cipher.decrypt_and_verify(ciphertext, tag)

    # Convert bytes back to numpy array
    decrypted_np_array = np.frombuffer(decrypted_data, dtype=dtype).reshape(shape)

    # Convert numpy array to jnp array
    decrypted_jnp_array = jnp.array(decrypted_np_array)

    return decrypted_jnp_array


# Example usage
if __name__ == "__main__":
    # Generate a random key for AES-256 (32 bytes)
    key = get_random_bytes(32)

    # Create a jnp array
    original_jnp_array = jnp.array(np.random.uniform(u_low, u_high, (m,)))

    # Encrypt the jnp array using AES-GCM
    ciphertext, tag, nonce = encrypt_jnp_array_gcm(original_jnp_array, key)

    # Decrypt to a jnp array
    decrypted_jnp_array = decrypt_to_jnp_array_gcm(
        ciphertext, tag, nonce, key, dtype=jnp.float32, shape=original_jnp_array.shape
    )

    # Check if the original and decrypted arrays are the same
    print("Original JAX Array:")
    print(original_jnp_array)
    print("\nDecrypted JAX Array:")
    print(decrypted_jnp_array)
    print(
        "\nArrays are equal:", jnp.array_equal(original_jnp_array, decrypted_jnp_array)
    )

Original JAX Array:
[1.8099765  1.1938232  0.5535451  0.1056134  1.9726875  0.13149214
 0.5494237  1.9824127  0.91129005 1.2849427  1.4183081  0.30188218
 0.16404134 1.0784012  1.2930766  0.4702789  0.7091443  0.38381267
 1.1181881  1.7664443  0.48831028 1.2204642  1.3051376  1.310405
 1.2894882  1.6884228  0.12458353 1.5344567  0.7271684  0.80978155
 0.09589244 0.87247145 0.0835437  1.5006124  0.31457186 0.680205
 0.33712184 0.3142509  1.7292604  0.11346394 1.1876285  0.69573295
 0.64739054 1.4297886  1.3745352  0.78202015 1.1897178  0.36977035
 0.7686286  0.6290839  1.9737662  0.04530505 1.0885407  0.591877
 0.1905414  0.38738957 1.6392007  1.6766782  0.53217536 0.5081069
 0.7968652  1.4661356  0.747126   1.8262628  0.39944172 0.35416263
 0.01446545 0.44505873 1.6425055  0.04913149 1.5593477  0.3170378
 1.9049246  1.612461   1.8463647  0.21917751 0.5966844  0.3406105
 1.1928747  1.3995585  0.86930627 0.9831869  1.8804462  0.56959033
 0.33260134 0.404725   1.8537736  0.6392202  0.6778

In [21]:
import numpy as np
import jax.numpy as jnp
import jax

# Simulate handles

handle_i_j = edge_tees[i](lambda x: get_random_bytes(x))(kappa)
handle_i_s = edge_tees[i](lambda x: get_random_bytes(x))(kappa)

# note that the establishment is not simplified.
handle_j_i = handle_i_j.to(edge_tees[j])
handle_j_s = edge_tees[j](lambda x: get_random_bytes(x))(kappa)

server_handle_i_s = handle_i_s.to(server_tee)
server_handle_j_s = handle_j_s.to(server_tee)


# P_i holds u_i
u_i = edge_devices[i](lambda x: x)(jnp.array(np.random.uniform(u_low, u_high, (m,))))

# P_j holds u_j
u_j = edge_devices[j](lambda x: x)(jnp.array(np.random.uniform(u_low, u_high, (m,))))


# corr preprocessing
def corr(k, m, dev1, key1, dev2, key2, return_zero_sharing=False):
    """Correlation function

    Args:
        k (int): ring size will be 2^k. Support k = 64 or 128 for now
        m (int): size of array to be correlated
        dev1 (Device): device 1
        key1 (Key): key for device 1
        dev2 (Device): device 2
        key2 (Key): key for device 2, key2 must be the same as key1 yet hold by different device
    """
    assert k == 32, "Only support k = 32 for now"
    dtype = jnp.uint32
    corr_dev1 = dev1(
        lambda key, shape, dtype: jax.random.bits(
            bytes_to_jax_random_key(key), shape, dtype
        )
    )(key1, (m,), dtype)
    if not return_zero_sharing:
        corr_dev2 = dev2(
            lambda key, shape, dtype: jax.random.bits(
                bytes_to_jax_random_key(key), shape, dtype
            )
        )(key2, (m,), dtype)
    else:
        corr_dev2 = dev2(
            lambda key, shape, dtype: -jax.random.bits(
                bytes_to_jax_random_key(key), shape, dtype
            )
        )(key2, (m,), dtype)
    return corr_dev1, corr_dev2


def cos_sim(u_i, u_j):
    # proramming details not related to protocol
    fixed_point_type = jnp.uint32
    shape_ref_i = edge_tees[i](lambda x: x.shape)(u_i.to(edge_tees[i]))
    shape_ref_j = edge_tees[j](lambda x: x.shape)(u_j.to(edge_tees[j]))
    # let's suppose that the reference type and shape are the same for u_i and u_j and it is ok to share to server
    shape_ref_server = shape_ref_i.to(server_tee)

    # preprocessing
    server_a, edge_tee_i_a = corr(
        k, m, server_tee, server_handle_i_s, edge_tees[i], handle_i_s
    )
    server_b, edge_tee_j_b = corr(
        k, m, server_tee, server_handle_j_s, edge_tees[j], handle_j_s
    )
    c = server_tee(lambda a, b: a * b)(server_a, server_b)

    # normalize u_i and u_j
    u_i_normalized = edge_tees[i](
        lambda x: fixed_point_type(x / jnp.linalg.norm(x) * (2.0**fxp))
    )(u_i.to(edge_tees[i]))
    u_j_normalized = edge_tees[j](
        lambda x: fixed_point_type(x / jnp.linalg.norm(x) * (2.0**fxp))
    )(u_j.to(edge_tees[j]))

    # E_i encrypts e = u_i_normalized - a, sends to P_j via P_i
    e = edge_tees[i](lambda x, y: x - y)(u_i_normalized, edge_tee_i_a)
    c_e, c_e_tag, c_e_nouce = edge_tees[i](encrypt_jnp_array_gcm, num_returns=3)(
        e, handle_i_j
    )
    c_e_j = c_e.to(edge_devices[i]).to(edge_devices[j])
    c_e_tag_j = c_e_tag.to(edge_devices[i]).to(edge_devices[j])
    c_e_nouce_j = c_e_nouce.to(edge_devices[i]).to(edge_devices[j])

    # E_j encrypts f = u_j_normalized - b, sends to P_i via P_j
    f = edge_tees[j](lambda x, y: x - y)(u_j_normalized, edge_tee_j_b)
    c_f, c_f_tag, c_f_nouce = edge_tees[j](encrypt_jnp_array_gcm, num_returns=3)(
        f, handle_j_i
    )

    c_f_i = c_f.to(edge_devices[j]).to(edge_devices[i])
    c_f_tag_i = c_f_tag.to(edge_devices[j]).to(edge_devices[i])
    c_f_nouce_i = c_f_nouce.to(edge_devices[j]).to(edge_devices[i])

    # P_i decrypts f in E_i
    try:
        f_dec = edge_tees[i](decrypt_to_jnp_array_gcm)(
            c_f_i.to(edge_tees[i]),
            c_f_tag_i.to(edge_tees[i]),
            c_f_nouce_i.to(edge_tees[i]),
            handle_i_j,
            fixed_point_type,
            shape_ref_i,
        )
        sf.wait(f_dec)
    except:
        raise RuntimeError("Error in decrypting f, abort")

    # P_i decrypts c in E_j
    try:
        e_dec = edge_tees[j](decrypt_to_jnp_array_gcm)(
            c_e_j.to(edge_tees[j]),
            c_e_tag_j.to(edge_tees[j]),
            c_e_nouce_j.to(edge_tees[j]),
            handle_j_i,
            fixed_point_type,
            shape_ref_j,
        )
        sf.wait(e_dec)
    except:
        raise RuntimeError("Error in decrypting c, abort")

    edge_tee_i_u_i_bracket_1, edge_tee_j_u_i_bracket_1 = corr(
        k, m, edge_tees[i], handle_i_j, edge_tees[j], handle_j_i
    )
    edge_tee_i_u_j_bracket_0, edge_tee_j_u_j_bracket_0 = corr(
        k, m, edge_tees[i], handle_i_j, edge_tees[j], handle_j_i
    )
    edge_tee_i_d, edge_tee_j_d = corr(
        k, m, edge_tees[i], handle_i_j, edge_tees[j], handle_j_i, True
    )

    u_i_bracket_0 = edge_tees[i](lambda x, y: x - y)(u_i, edge_tee_i_u_i_bracket_1)
    u_j_bracket_1 = edge_tees[j](lambda x, y: x - y)(u_j, edge_tee_j_u_j_bracket_0)

    # E_i computes:
    z_bracket_0 = edge_tees[i](
        lambda x1, x2, x3, x4, x5: fixed_point_type(
            fixed_point_type(x1 * (x2 / (2.0**fxp)))
            + fixed_point_type(x3 * (x4 / (2.0**fxp)))
            + fixed_point_type(x1 * (x3 / (2.0**fxp)))
            + x5
        )
    )(e, edge_tee_i_u_j_bracket_0, f_dec, u_i_bracket_0, edge_tee_i_d)

    # E_i encrypts z_bracket_0, sends to server tee via server
    c_z_bracket_0, c_z_bracket_0_tag, c_z_bracket_0_nouce = edge_tees[i](
        encrypt_jnp_array_gcm, num_returns=3
    )(z_bracket_0, handle_i_s)
    c_z_bracket_0_server = c_z_bracket_0.to(server_device).to(server_tee)
    c_z_bracket_0_tag_server = c_z_bracket_0_tag.to(server_device).to(server_tee)
    c_z_bracket_0_nouce_server = c_z_bracket_0_nouce.to(server_device).to(server_tee)

    # E_j computes:
    z_bracket_1 = edge_tees[j](
        lambda x1, x2, x3, x4, x5: fixed_point_type(
            fixed_point_type(x1 * (x2 / (2.0**fxp))) + fixed_point_type(x3 * (x4 / (2.0**fxp))) + x5
        )
    )(e_dec, u_j_bracket_1, f, edge_tee_j_u_i_bracket_1, edge_tee_j_d)

    # E_j encrypts z_bracket_1, sends to server tee via server
    c_z_bracket_1, c_z_bracket_1_tag, c_z_bracket_1_nouce = edge_tees[j](
        encrypt_jnp_array_gcm, num_returns=3
    )(z_bracket_1, handle_j_s)
    c_z_bracket_1_server = c_z_bracket_1.to(server_device).to(server_tee)
    c_z_bracket_1_tag_server = c_z_bracket_1_tag.to(server_device).to(server_tee)
    c_z_bracket_1_nouce_server = c_z_bracket_1_nouce.to(server_device).to(server_tee)

    # server tries to decrypt
    try:
        z_bracket_0_dec = server_tee(decrypt_to_jnp_array_gcm)(
            c_z_bracket_0_server,
            c_z_bracket_0_tag_server,
            c_z_bracket_0_nouce_server,
            server_handle_i_s,
            fixed_point_type,
            shape_ref_server,
        )
        z_bracket_1_dec = server_tee(decrypt_to_jnp_array_gcm)(
            c_z_bracket_1_server,
            c_z_bracket_1_tag_server,
            c_z_bracket_1_nouce_server,
            server_handle_j_s,
            fixed_point_type,
            shape_ref_server,
        )
        sf.wait([z_bracket_0_dec, z_bracket_1_dec])
    except:
        raise Exception("Decryption z values failed")

    z = server_tee(lambda x, y: x + y)(z_bracket_0_dec, z_bracket_1_dec)
    cos_sim_val = server_tee(lambda x, y: jnp.sum(fixed_point_type(x + y) / (2.0**fxp)))(z, c)
    return cos_sim_val

In [22]:
sf.reveal(cos_sim(u_i, u_j))

Array(3536.3691, dtype=float32)

In [23]:
# import required libraries
import numpy as np
from numpy.linalg import norm

# define two lists or array
A = sf.reveal(u_i)
B = sf.reveal(u_j)

print("A:", A)
print("B:", B)

# compute cosine similarity
cosine = np.dot(A, B) / (norm(A) * norm(B))
print("Cosine Similarity:", cosine)

A: [0.83418036 1.8201618  1.5297189  1.2890375  0.73208886 1.731295
 0.3401033  1.4305145  0.94484836 0.46247536 1.3172956  1.671017
 1.594989   1.1906219  0.802751   1.7769768  0.27613756 0.5903945
 0.22402883 0.43292436 0.26687616 0.6569454  0.39032975 0.16447274
 0.34521627 0.536313   0.70829433 0.3660815  0.80461323 1.8812189
 1.5043143  1.2484449  0.978908   0.00760418 0.7296152  1.3250883
 1.2293415  0.20926672 0.20755905 0.1252091  0.49740764 0.01567223
 0.34565863 1.9476775  0.9103757  0.08338346 1.929178   1.0823679
 0.38455954 0.35360786 1.0640564  1.4298433  0.5540249  0.79089826
 1.5828185  1.182275   1.3900359  0.9434403  0.26861718 0.29207328
 1.8017027  0.04550739 0.9350107  1.1293414  1.030083   1.7489316
 1.8103809  1.643495   0.960025   1.9701657  1.0078423  0.9125273
 1.2525904  1.2901816  0.04048604 0.0567827  1.9795396  0.12273638
 0.34115297 0.51150656 1.5127676  0.9733286  0.39797193 0.67661285
 1.4936466  1.0287958  1.6157407  0.64581865 1.4148384  0.09941606
 1